In [1]:
import sqlite3
import pandas as pd
from sqlite3 import Error
from matplotlib import pyplot as plt
import math

In [2]:
def create_connection(db_file):    
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("connexion établie")
        return conn
    except Error as e:
        print(e)

    return conn

In [3]:
with open('orders.sql', 'r') as sql_file:
    sql_script = sql_file.read()

In [ ]:
conn = sqlite3.connect('../data/marketplaces.db')
cur = conn.cursor()
cur.executescript(sql_script)

La partie ci-dessus servait à nettoyer la base de données, et la partie suivante sert pour l'analyse.

In [15]:
# get rounded coordinates of each zip codes
cur.execute('''
            SELECT geolocation_zip_code_prefix, AVG(geolocation_lat), AVG(geolocation_lng)
            FROM geolocations
            GROUP BY geolocation_zip_code_prefix
            ''')
coordinates_df = pd.DataFrame(cur.fetchall(), columns = ['Zip', 'Latitude', 'Longitude'])

# get the rest
cur.execute('''
            SELECT DISTINCT
            price, freight_value,
            (JULIANDAY(order_delivered_customer_date)
            - JULIANDAY(order_delivered_carrier_date)), seller_zip_code_prefix, customer_zip_code_prefix
            FROM items i
            INNER JOIN orders o ON i.order_id = o.order_id
            INNER JOIN sellers s ON s.seller_id = i.seller_id
            INNER JOIN customers c ON c.customer_id = o.customer_id
            ''')
shipping_df = pd.DataFrame(cur.fetchall(), columns = ['Price', 'Freight Value',
                                                      'Delivery Duration', 'Seller Zip', 'Customer Zip'])

# merge info with proper zip coordinates
shipping_df = pd.merge(shipping_df, coordinates_df, how='left', left_on='Seller Zip', right_on='Zip')
shipping_df = pd.merge(shipping_df, coordinates_df, how='left', left_on='Customer Zip', right_on='Zip')
shipping_df = shipping_df.drop(columns=['Zip_x', 'Zip_y'])
shipping_df = shipping_df.rename(columns={
                                          'Latitude_x':'Seller Latitude',
                                          'Longitude_x':'Seller Longitude',
                                          'Latitude_y':'Customer Latitude',
                                          'Longitude_y':'Customer Longitude'
                                         })

# using pythagoras to calculate distance
shipping_df['Delivery Distance'] = (((shipping_df['Customer Latitude'] -
                                      shipping_df['Seller Latitude']) ** 2) +
                                    ((shipping_df['Customer Longitude'] -
                                      shipping_df['Seller Longitude']) ** 2)) ** 0.5
shipping_df = shipping_df.drop(columns=['Seller Zip', 'Customer Zip',
                                        'Seller Latitude', 'Customer Latitude',
                                        'Seller Longitude', 'Customer Longitude'])

# printing and plotting correlation
print(shipping_df)
corr = shipping_df.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

        Price  Freight Value  Delivery Duration  Delivery Distance
0       58.90          13.29           1.214954           2.911114
1      239.90          19.93           8.062083           5.510030
2      199.00          17.87           6.029491           2.866108
3       12.99          12.79           4.003229           2.673821
4      199.90          18.14          13.289144           6.315937
...       ...            ...                ...                ...
99351  299.99          43.41          15.449769          24.802057
99352  350.00          36.53           6.518692           3.364803
99353   99.90          16.95           1.881343           3.240030
99354   55.99           8.72           1.122766           0.661899
99355   43.00          12.79           3.156551           1.222741

[99356 rows x 4 columns]


,Price,Freight Value,Delivery Duration,Delivery Distance
Price,1.000000,0.419034,0.038765,0.085126
Freight Value,0.419034,1.000000,0.195990,0.394528
Delivery Duration,0.038765,0.195990,1.000000,0.422460
Delivery Distance,0.085126,0.394528,0.422460,1.000000


D'après la table de coorélation ci-dessus, la durée de livraison ne semble pas influer sur les frais de port, néanmoins il existe une petite influence de par la distance de livraison et par le prix initial.
Mais voyons plutôt si les dimensions du produits ne sont pas plus pertinents.

In [13]:
cur.execute('''
            SELECT DISTINCT
            price, freight_value, product_weight_g, product_length_cm, product_height_cm, product_width_cm
            FROM items INNER JOIN products ON items.product_id = products.product_id
            ''')
dimension_df = pd.DataFrame(cur.fetchall(), columns = ['Price', 'Freight Value',
                                                       'Product Weight (g)', 'Product Length (cm)',
                                                       'Product Height (cm)', 'Product Width (cm)'])

# printing and plotting correlation
print(dimension_df)
corr = dimension_df.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

        Price  Freight Value  Product Weight (g)  Product Length (cm)  \
0       58.90          13.29               650.0                 28.0   
1      239.90          19.93             30000.0                 50.0   
2      199.00          17.87              3050.0                 33.0   
3       12.99          12.79               200.0                 16.0   
4      199.90          18.14              3750.0                 35.0   
...       ...            ...                 ...                  ...   
71947    4.90          10.96               100.0                 60.0   
71948   17.90          22.06               200.0                 25.0   
71949  299.99          43.41             10150.0                 89.0   
71950  350.00          36.53              8950.0                 45.0   
71951   43.00          12.79               600.0                 30.0   

       Product Height (cm)  Product Width (cm)  
0                      9.0                14.0  
1                     30.

,Price,Freight Value,Product Weight (g),Product Length (cm),Product Height (cm),Product Width (cm)
Price,1.000000,0.410341,0.336280,0.148860,0.224015,0.174587
Freight Value,0.410341,1.000000,0.613942,0.327809,0.392811,0.338719
Product Weight (g),0.336280,0.613942,1.000000,0.479332,0.579340,0.518125
Product Length (cm),0.148860,0.327809,0.479332,1.000000,0.200247,0.532906
Product Height (cm),0.224015,0.392811,0.579340,0.200247,1.000000,0.294112
Product Width (cm),0.174587,0.338719,0.518125,0.532906,0.294112,1.000000


Sur cette table, on peut voir des relations déjà plus concrètes. Le poids du produit semble être le paramêtre le plus en rapport avec les frais de ports.